Julia code for Kitaev model.
\section{Kitaev model without symmetry}

In [21]:
function index(N1, N2, c, r, atom)
    #Calculate the index of the atoms in the (r,c) unit cell
    c = (c + N1) % N1 
    r = (r + N2) % N2 
    #println("(",r,",",c,")")
    n = r * N1 + c 
    if atom == 'A'
        ind = 2 * n + 1
    elseif atom == 'B'
        ind = 2 * n + 1 + 1
    else
        return println("ERROR: The atom index should be A or B.")
    return ind
    end
end

function StateConfig(tag, N1, N2)
    #=The configuration of the state with tag.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers       
    Outputs: binary form of tag. type: 1D array=#
    
    dim = 2^(N1 * N2 * 2)
    if tag >= dim
        return println("Error: The tag is out of range.")
    else
        b = string(tag, base = 2, pad = N1 * N2 * 2)
        return b
    end
end

function flip(tag, i, j, N1, N2)
    #=Flip the spin on i,j site.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers
            i,j: position of spins that are flipped
    Output: The tag of new state, type: int =#
    SiteNum = N1 * N2 * 2
    f = 2^(SiteNum - i) + 2^(SiteNum - j)
    return ntag = xor(tag,f)
end

function KitaevRhom(J, N1, N2)
    #=Kitaev Hamiltonian on N1 * N2 * 2 lattice
    Inputes: N1, N2: column and row numbers
             J: coupling constants
    Outpus: The Non-zero Hamiltonian element H[i] is in the 
            position (row[i], col[i])
            H, row, col : type: list =#
    SiteNum = N1 * N2 * 2
    dim = 2^SiteNum
    
    numNZero = 0  #count the number of non-zero element
    data = Float64[]
    row = Int[]
    col = Int[]
    for tag = 0: dim-1
        #println("The state numnber is: " , tag)
        si = StateConfig(tag, N1, N2) # initial state configuration
        #println("initial configuration=",si,", tag = ",tag)
        
        temp = Dict{Int, Float64}() # A temporay dict
        
        for r = 0: N2-1
            for c = 0: N1-1
                ind = index(N1, N2, c, r, 'A')
                # Sx Sx
                next = index(N1, N2, c, r, 'B')
                
                ntag = flip(tag, ind, next, N1, N2)
                if ntag in keys(temp)
                    temp[ntag] += -J[1] /4
                else
                    temp[ntag] = -J[1] /4
                end  
                
                # Sy Sy
                next = index(N1, N2, c - 1, r, 'B')
                
                ntag = flip(tag, ind, next, N1, N2) 
                if ntag in keys(temp)
                    temp[ntag] += J[2] * (parse(Int,si[ind]) - 0.5) * (parse(Int,si[next]) - 0.5)
                else
                    temp[ntag] = J[2] * (parse(Int,si[ind]) - 0.5) * (parse(Int,si[next]) - 0.5)
                end
                        
                #Sz Sz
                next = index(N1, N2, c, r - 1, 'B')
                    
                if tag in keys(temp)
                    temp[tag] += - J[3] * (parse(Int,si[ind]) - 0.5) * (parse(Int,si[next]) - 0.5)
                else
                    temp[tag] = - J[3] * (parse(Int,si[ind]) - 0.5)* (parse(Int,si[next]) - 0.5)
                end
            end
        end
                        
        odTemp = sort(collect(keys(temp)))
        for sf in odTemp
            append!(col, [tag + 1])
            append!(row, [sf + 1])
            append!(data,[temp[sf]])
            numNZero += 1
        end
    end
    
    println("non zero elemens number =" , numNZero)
    println("Percentage =" , numNZero/(dim * dim) * 100,"%")
    return col, row, data
end



KitaevRhom (generic function with 1 method)

In [ ]:
N1 = 3
N2 = 4
J = [1,1,1]

col, row, data = KitaevRhom(J, N1, N2)

using SparseArrays
#dim = 2^(N1 * N2 * 2)
H = sparse(row, col, data)

using Arpack
e,x = eigs(H)
println(e/(N1*N2*2))

\section{Kitaev model with translation symmetry}

In [ ]:
function index(N1::Int, N2::Int, c::Int, r::Int, atom)
    #Calculate the index of the atoms in the (r,c) unit cell
    c = (c + N1) % N1 
    r = (r + N2) % N2 
    #println("(",r,",",c,")")
    n = r * N1 + c 
    if atom == 'A'
        ind = 2 * n + 1
    elseif atom == 'B'
        ind = 2 * n + 1 + 1
    else
        return println("ERROR: The atom index should be A or B.")
    return ind
    end
end

function StateConfig(tag, N1, N2)
    #=The configuration of the state with tag.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers       
    Outputs: binary form of tag. type: 1D array=#
    
    dim = 2^(N1 * N2 * 2)
    if tag >= dim
        return println("Error: The tag is out of range.")
    else
        b = string(tag, base = 2, pad = N1 * N2 * 2)
        return b
    end
end

function flip(tag, i, j, N1, N2)
    #=Flip the spin on i,j site.
    Inputs: tag: tag of a state
            N1, N2: column and row numbers
            i,j: position of spins that are flipped
    Output: The tag of new state, type: int =#
    SiteNum = N1 * N2 * 2
    f = 2^(SiteNum - i) + 2^(SiteNum - j)
    return ntag = xor(tag,f)
end

function cyclebits(n1::Int,n2::Int, tag::Int, N1::Int, N2::Int)
    #=Performs a cyclic permutations of n1 steps to the right and n2 steps upward.
       Inputs: tag: tag of the reference state
               n1: cycle times of n1-direction, columns,if n1=0, no cycle in columns
               n2: cycle times of n2-direction, rows,if n2=0, no cycle in rows
               N1: number of columns
               N2: number of rows
       Outputs: ntag: the tag of the outcoming state=#
    dim = N1 * N2
    SiteNum = N1 * N2 * 2
    config = StateConfig(tag, N1, N2)
    sub = list(config[::2] + config[1::2])
    a = np.array(sub).reshape(2,N2,N1)

    # cycle of columns
    Nsub = []
    #Nsub_for = []
    if n1 != 0:
        for n in range(2):
            b = a[n,:,:].tolist()
            for i in range(N2):
                b[i] = b[i][n1:] + b[i][:n1]
            Nsub += list(_flatten(b))
            #print('Nsub = ', Nsub)
            
            #for i in range(N2):
                #for j in range(N1):
                    #Nsub_for += b[i][j]
            #print('Nsub_for = ', Nsub_for)
            
    #cycle of rows
    if n2 != 0:
        for n in range(2):
            at = a[n,:,:].T 
            b = at.tolist()
            for i in range(N1):
                b[i] = b[i][n2:] + b[i][:n2]
            na = np.array(b).T      
            nb = na.tolist()
            Nsub += list(_flatten(nb))
            #print('Nsub = ', Nsub)
            
            #for i in range(N2):
                #for j in range(N1):
                    #Nsub_for += b[j][i]
            #print('Nsub_for = ', Nsub_for)
            
    # Nsub = [A,A,A,B,B,B], Nconfig = [A,B,A,B,A,B]
    Nconfig = list(itertools.chain.from_iterable(zip(Nsub[:dim],Nsub[dim:])))
    Nconfig_str = "".join(Nconfig)
    
    #Nconfig_str = "".join(Nsub)
    #print(Nconfig_str)
    ntag = int(Nconfig_str, 2)    
    return ntag
